In [1]:
var('t q')

def number_of_ones(partition):
    for i in range(len(partition)):
        if partition[len(partition) - i - 1] != 1:
            return i
    return len(partition)

def my_len(partition):
    return 2*len(partition) - number_of_ones(partition)

def series_t_q(list_of_list_of_partition):
    series = 0
    for i in range(len(list_of_list_of_partition)):
        series_t = 0
        for j in range(len(list_of_list_of_partition[i])):
            series_t = series_t + t^my_len(list_of_list_of_partition[i][j])
        series = series + series_t*q^i
    return series

def partition_ends(partition, tail, is_greater):
    if len(partition) < len(tail) - 1:
        return False
    partition2 = partition[len(partition) - len(tail) + 1: len(partition)]
    if partition2 != tail[1:]:
        return False
    if is_greater:
        if partition == partition2:
            return True
        if partition[len(partition) - len(tail)] > tail[0]:
            return True
    else:
        if partition == partition2:
            return False
        if partition[len(partition) - len(tail)] == tail[0]:
            return True
    return False

def list_of_list_of_partition_ends(list_of_list_of_partition, tail, is_greater):
    return [[partition
             for partition in list_of_partition if partition_ends(partition, tail, is_greater)]
            for list_of_partition in list_of_list_of_partition]

def vector_sum(vector1, vector2):
    return {partition: vector1.get(partition, 0) + vector2.get(partition, 0)
            for partition in set(vector1)|set(vector2)}

def vector_simplify0(partition):
    for i in range(len(partition) - 1):
        if partition[i] < partition[i + 1]:
            return {partition[0: i] + (partition[i] + partition[i + 1],)
                    + partition[i + 2: len(partition)]: partition[i + 1] - partition[i],
                    partition[0: i] + (partition[i + 1],) + (partition[i],)
                    + partition[i + 2: len(partition)]: 1}
    return {partition: 1}

def vector_simplify(vector):
    vector2 = {}
    for partition in set(vector):
        vector3 = vector_simplify0(partition)
        vector2 = vector_sum(vector2, {partition2: vector3[partition2]*vector[partition]
                                       for partition2 in set(vector3)})
    if vector2 == vector:
        return vector2
    return vector_simplify(vector2)

def L_partition_vector(partition, vector):
    return vector_simplify(
        {partition + partition2: vector[partition2] for partition2 in set(vector)})

def my_order(a):
    polynomial = to_polynomial(a)
    polynomial = [-k for k in polynomial]
    polynomial.reverse()
    return ((my_len(a), len(a) - number_of_ones(a), polynomial)) 

def ordered_partitions(n):
    if n <= 0:
        return [[]]
    list_of_partition = list(Partitions(n))
    list_of_partition.sort(key = my_order, reverse = True)
    return list_of_partition

def is_subpartition(partition, partition2):
    if len(partition) < len(partition2):
        return False
    for i in range(len(partition) - len(partition2) + 1):
        if partition[i: i + len(partition2)] == partition2:
            return True
    return False

def partition_contains_any_of_list(partition, list_of_partition):
    for partition2 in list_of_partition:
        if is_subpartition(partition, partition2):
            return True
    return False

def partition_contains_any_of_Q(partition):
    if partition == []:
        return False
    if partition_contains_any_of_list(
        partition,
        [[2], [1, 1, 1], [3, 1, 1], [3, 3], [4, 3, 1], [4, 4, 1], [5, 4, 1, 1], [6, 5, 3, 1]]):
        return True
    if len(partition) > 2:
        for i in range(len(partition) - 2):
            if (partition[i] - partition[i + 2] <= 2 and partition[i + 2] > 1):
                return True
    if len(partition) > 3:
        for i in range(len(partition) - 3):
            if ((partition[i] - partition[i + 3] <= 3 and partition[i + 3] > 1)
                or (partition[i] - partition[i + 3] <= 4
                    and partition[i + 1] - partition[i + 2] <= 3 and partition[i + 3] > 1)):
                return True
    if len(partition) > 4:
        for i in range(len(partition) - 4):
            if (partition[i] - partition[i + 1] == 1 and partition[i + 1] - partition[i + 2] == 2
                and partition[i + 2] - partition[i + 3] == 2
                and partition[i + 3] - partition[i + 4] == 1 and partition[i + 4] > 1):
                return True
    return False

def matrix_UVir_m2_m3(n):
    m2 = {(1, 1): -3, (2,): 4}
    m3 = {(2, 1): -12, (1, 1, 1): 4, (3,): 3}
    list_of_partition = ordered_partitions(n)
    list_of_partition2 = ordered_partitions(n - 2)
    list_of_partition4 = ordered_partitions(n - 3)
    list_of_vector_m2 = [L_partition_vector(tuple(list_of_partition2[i]), m2)
                         for i in range(len(list_of_partition2))]
    list_of_vector_m3 = [L_partition_vector(tuple(list_of_partition4[i]), m3)
                         for i in range(len(list_of_partition4))]
    matrix_m2 = matrix({(i, j): list_of_vector_m2[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition2))}, sparse = False)
    matrix_m3 = matrix({(i, j): list_of_vector_m3[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition4))}, sparse = False)
    matrix_m2 = matrix_m2.stack(matrix_m3)
    matrix_m2 = matrix_m2.rref()
    return matrix_m2

def to_polynomial(partition):
    polynomial = [0]*(sum(partition) - 1)
    for i in range(len(partition) - number_of_ones(partition)):
        polynomial[partition[i] - 2] += 1
    return polynomial

def pivots_up_to(n):
    list_of_list_of_partition = []
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [list_of_partition[j] for j in matrix_UVir_m2_m3(i).pivots()])
    return list_of_list_of_partition

def basis_up_to(n):
    list_of_list_of_partition = []
    pivots = pivots_up_to(n)
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition if partition not in pivots[i]])
    return list_of_list_of_partition

def basis_up_tov2(n):
    list_of_list_of_partition = []
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition
             if not partition_contains_any_of_Q(partition)])
    return list_of_list_of_partition

In [2]:
%time w = basis_up_to(29)

CPU times: user 9min 21s, sys: 36.4 s, total: 9min 57s
Wall time: 9min 58s


In [3]:
w

[[[]],
 [[1]],
 [[1, 1]],
 [[3]],
 [[3, 1], [4]],
 [[4, 1], [5]],
 [[4, 1, 1], [5, 1], [6]],
 [[4, 3], [5, 1, 1], [6, 1], [7]],
 [[4, 4], [5, 3], [6, 1, 1], [7, 1], [8]],
 [[5, 3, 1], [5, 4], [6, 3], [7, 1, 1], [8, 1], [9]],
 [[5, 4, 1], [6, 3, 1], [5, 5], [6, 4], [7, 3], [8, 1, 1], [9, 1], [10]],
 [[5, 5, 1],
  [6, 4, 1],
  [7, 3, 1],
  [6, 5],
  [7, 4],
  [8, 3],
  [9, 1, 1],
  [10, 1],
  [11]],
 [[5, 5, 1, 1],
  [6, 4, 1, 1],
  [6, 5, 1],
  [7, 4, 1],
  [8, 3, 1],
  [6, 6],
  [7, 5],
  [8, 4],
  [9, 3],
  [10, 1, 1],
  [11, 1],
  [12]],
 [[6, 4, 3],
  [6, 5, 1, 1],
  [7, 4, 1, 1],
  [6, 6, 1],
  [7, 5, 1],
  [8, 4, 1],
  [9, 3, 1],
  [7, 6],
  [8, 5],
  [9, 4],
  [10, 3],
  [11, 1, 1],
  [12, 1],
  [13]],
 [[6, 5, 3],
  [7, 4, 3],
  [6, 6, 1, 1],
  [7, 5, 1, 1],
  [8, 4, 1, 1],
  [7, 6, 1],
  [8, 5, 1],
  [9, 4, 1],
  [10, 3, 1],
  [7, 7],
  [8, 6],
  [9, 5],
  [10, 4],
  [11, 3],
  [12, 1, 1],
  [13, 1],
  [14]],
 [[6, 6, 3],
  [7, 4, 4],
  [7, 5, 3],
  [8, 4, 3],
  [7, 6, 1, 1],
 

In [4]:
w == basis_up_tov2(29)

True

In [5]:
%time series_t_q(basis_up_tov2(60))

CPU times: user 5min 27s, sys: 10.9 s, total: 5min 38s
Wall time: 5min 38s


(2*t^15 + 156*t^14 + 463*t^13 + 1569*t^12 + 1212*t^11 + 2169*t^10 + 819*t^9 + 1114*t^8 + 229*t^7 + 266*t^6 + 27*t^5 + 29*t^4 + t^3 + t^2)*q^60 + (108*t^14 + 377*t^13 + 1344*t^12 + 1087*t^11 + 1980*t^10 + 770*t^9 + 1048*t^8 + 220*t^7 + 256*t^6 + 27*t^5 + 28*t^4 + t^3 + t^2)*q^59 + (74*t^14 + 300*t^13 + 1151*t^12 + 974*t^11 + 1806*t^10 + 718*t^9 + 990*t^8 + 212*t^7 + 247*t^6 + 26*t^5 + 28*t^4 + t^3 + t^2)*q^58 + (47*t^14 + 237*t^13 + 974*t^12 + 867*t^11 + 1640*t^10 + 673*t^9 + 929*t^8 + 203*t^7 + 238*t^6 + 26*t^5 + 27*t^4 + t^3 + t^2)*q^57 + (30*t^14 + 182*t^13 + 822*t^12 + 771*t^11 + 1488*t^10 + 625*t^9 + 876*t^8 + 195*t^7 + 229*t^6 + 25*t^5 + 27*t^4 + t^3 + t^2)*q^56 + (16*t^14 + 140*t^13 + 684*t^12 + 680*t^11 + 1344*t^10 + 584*t^9 + 819*t^8 + 187*t^7 + 220*t^6 + 25*t^5 + 26*t^4 + t^3 + t^2)*q^55 + (9*t^14 + 102*t^13 + 569*t^12 + 599*t^11 + 1212*t^10 + 540*t^9 + 770*t^8 + 179*t^7 + 212*t^6 + 24*t^5 + 26*t^4 + t^3 + t^2)*q^54 + (4*t^14 + 74*t^13 + 463*t^12 + 523*t^11 + 1087*t^10 + 503*t

In [6]:
series_t_q(w)

(2*t^10 + 19*t^9 + 58*t^8 + 35*t^7 + 51*t^6 + 12*t^5 + 13*t^4 + t^3 + t^2)*q^29 + (t^10 + 13*t^9 + 49*t^8 + 32*t^7 + 47*t^6 + 11*t^5 + 13*t^4 + t^3 + t^2)*q^28 + (9*t^9 + 39*t^8 + 28*t^7 + 43*t^6 + 11*t^5 + 12*t^4 + t^3 + t^2)*q^27 + (5*t^9 + 32*t^8 + 25*t^7 + 39*t^6 + 10*t^5 + 12*t^4 + t^3 + t^2)*q^26 + (3*t^9 + 24*t^8 + 22*t^7 + 35*t^6 + 10*t^5 + 11*t^4 + t^3 + t^2)*q^25 + (t^9 + 19*t^8 + 19*t^7 + 32*t^6 + 9*t^5 + 11*t^4 + t^3 + t^2)*q^24 + (13*t^8 + 16*t^7 + 28*t^6 + 9*t^5 + 10*t^4 + t^3 + t^2)*q^23 + (9*t^8 + 14*t^7 + 25*t^6 + 8*t^5 + 10*t^4 + t^3 + t^2)*q^22 + (5*t^8 + 11*t^7 + 22*t^6 + 8*t^5 + 9*t^4 + t^3 + t^2)*q^21 + (3*t^8 + 9*t^7 + 19*t^6 + 7*t^5 + 9*t^4 + t^3 + t^2)*q^20 + (t^8 + 7*t^7 + 16*t^6 + 7*t^5 + 8*t^4 + t^3 + t^2)*q^19 + (5*t^7 + 14*t^6 + 6*t^5 + 8*t^4 + t^3 + t^2)*q^18 + (3*t^7 + 11*t^6 + 6*t^5 + 7*t^4 + t^3 + t^2)*q^17 + (2*t^7 + 9*t^6 + 5*t^5 + 7*t^4 + t^3 + t^2)*q^16 + (7*t^6 + 5*t^5 + 6*t^4 + t^3 + t^2)*q^15 + (5*t^6 + 4*t^5 + 6*t^4 + t^3 + t^2)*q^14 + (3*t^6 +

In [7]:
series_t_q(list_of_list_of_partition_ends(w, [2], True))

(35*t^8 + 41*t^6 + 12*t^4 + t^2)*q^29 + (29*t^8 + 37*t^6 + 12*t^4 + t^2)*q^28 + (22*t^8 + 34*t^6 + 11*t^4 + t^2)*q^27 + (17*t^8 + 30*t^6 + 11*t^4 + t^2)*q^26 + (12*t^8 + 27*t^6 + 10*t^4 + t^2)*q^25 + (9*t^8 + 24*t^6 + 10*t^4 + t^2)*q^24 + (5*t^8 + 21*t^6 + 9*t^4 + t^2)*q^23 + (3*t^8 + 18*t^6 + 9*t^4 + t^2)*q^22 + (t^8 + 16*t^6 + 8*t^4 + t^2)*q^21 + (13*t^6 + 8*t^4 + t^2)*q^20 + (11*t^6 + 7*t^4 + t^2)*q^19 + (9*t^6 + 7*t^4 + t^2)*q^18 + (7*t^6 + 6*t^4 + t^2)*q^17 + (5*t^6 + 6*t^4 + t^2)*q^16 + (4*t^6 + 5*t^4 + t^2)*q^15 + (2*t^6 + 5*t^4 + t^2)*q^14 + (t^6 + 4*t^4 + t^2)*q^13 + (4*t^4 + t^2)*q^12 + (3*t^4 + t^2)*q^11 + (3*t^4 + t^2)*q^10 + (2*t^4 + t^2)*q^9 + (2*t^4 + t^2)*q^8 + (t^4 + t^2)*q^7 + q^6*t^2 + q^5*t^2 + q^4*t^2 + q^3*t^2 + 1

In [8]:
series_t_q(list_of_list_of_partition_ends(w, [4], True))

(3*t^8 + 22*t^6 + 10*t^4 + t^2)*q^29 + (2*t^8 + 19*t^6 + 10*t^4 + t^2)*q^28 + (17*t^6 + 9*t^4 + t^2)*q^27 + (14*t^6 + 9*t^4 + t^2)*q^26 + (12*t^6 + 8*t^4 + t^2)*q^25 + (10*t^6 + 8*t^4 + t^2)*q^24 + (8*t^6 + 7*t^4 + t^2)*q^23 + (6*t^6 + 7*t^4 + t^2)*q^22 + (5*t^6 + 6*t^4 + t^2)*q^21 + (3*t^6 + 6*t^4 + t^2)*q^20 + (2*t^6 + 5*t^4 + t^2)*q^19 + (t^6 + 5*t^4 + t^2)*q^18 + (4*t^4 + t^2)*q^17 + (4*t^4 + t^2)*q^16 + (3*t^4 + t^2)*q^15 + (3*t^4 + t^2)*q^14 + (2*t^4 + t^2)*q^13 + (2*t^4 + t^2)*q^12 + (t^4 + t^2)*q^11 + (t^4 + t^2)*q^10 + q^9*t^2 + q^8*t^2 + q^7*t^2 + q^6*t^2 + q^5*t^2 + 1

In [9]:
series_t_q(list_of_list_of_partition_ends(w, [5, 4], True))

(5*t^8 + 7*t^6 + t^4)*q^29 + (4*t^8 + 7*t^6 + t^4)*q^28 + (2*t^8 + 6*t^6 + t^4)*q^27 + (t^8 + 6*t^6 + t^4)*q^26 + (5*t^6 + t^4)*q^25 + (5*t^6 + t^4)*q^24 + (4*t^6 + t^4)*q^23 + (4*t^6 + t^4)*q^22 + (3*t^6 + t^4)*q^21 + (3*t^6 + t^4)*q^20 + (2*t^6 + t^4)*q^19 + q^16*t^4 + (2*t^6 + t^4)*q^18 + q^15*t^4 + (t^6 + t^4)*q^17 + q^14*t^4 + q^13*t^4 + q^12*t^4 + q^11*t^4 + q^10*t^4 + q^4*t^2

In [10]:
series_t_q(list_of_list_of_partition_ends(w, [5, 4], False))

(4*t^8 + t^6)*q^29 + q^24*t^6 + (3*t^8 + t^6)*q^28 + q^23*t^6 + (3*t^8 + t^6)*q^27 + q^22*t^6 + (2*t^8 + t^6)*q^26 + q^21*t^6 + (t^8 + t^6)*q^25 + q^20*t^6 + q^19*t^6 + q^18*t^6 + q^17*t^6 + q^16*t^6 + q^9*t^4

In [11]:
series_t_q(list_of_list_of_partition_ends(w, [4, 4], False))

(4*t^8 + t^6)*q^29 + (4*t^8 + t^6)*q^28 + q^23*t^6 + (3*t^8 + t^6)*q^27 + q^22*t^6 + (3*t^8 + t^6)*q^26 + q^21*t^6 + (2*t^8 + t^6)*q^25 + q^20*t^6 + (2*t^8 + t^6)*q^24 + q^19*t^6 + q^18*t^6 + q^17*t^6 + q^16*t^6 + q^15*t^6 + q^8*t^4

In [12]:
series_t_q(list_of_list_of_partition_ends(w, [5, 3], True))

(8*t^8 + 8*t^6 + t^4)*q^29 + (6*t^8 + 7*t^6 + t^4)*q^28 + (5*t^8 + 7*t^6 + t^4)*q^27 + (3*t^8 + 6*t^6 + t^4)*q^26 + (2*t^8 + 6*t^6 + t^4)*q^25 + (t^8 + 5*t^6 + t^4)*q^24 + (5*t^6 + t^4)*q^23 + (4*t^6 + t^4)*q^22 + (4*t^6 + t^4)*q^21 + (3*t^6 + t^4)*q^20 + (3*t^6 + t^4)*q^19 + (2*t^6 + t^4)*q^18 + (2*t^6 + t^4)*q^17 + q^14*t^4 + (t^6 + t^4)*q^16 + q^13*t^4 + (t^6 + t^4)*q^15 + q^12*t^4 + q^11*t^4 + q^10*t^4 + q^9*t^4 + q^3*t^2

In [13]:
series_t_q(list_of_list_of_partition_ends(w, [6, 5, 3], True))

(4*t^8 + t^6)*q^29 + (4*t^8 + t^6)*q^28 + (3*t^8 + t^6)*q^27 + q^22*t^6 + (3*t^8 + t^6)*q^26 + q^21*t^6 + (2*t^8 + t^6)*q^25 + q^20*t^6 + (2*t^8 + t^6)*q^24 + q^19*t^6 + (t^8 + t^6)*q^23 + q^18*t^6 + q^17*t^6 + q^16*t^6 + q^15*t^6 + q^8*t^4

In [14]:
series_t_q(list_of_list_of_partition_ends(w, [6, 5, 3], False))

q^29*t^8 + q^28*t^8 + q^27*t^8 + q^26*t^8 + q^25*t^8 + q^24*t^8 + q^23*t^8 + q^22*t^8 + q^14*t^6

In [15]:
series_t_q(list_of_list_of_partition_ends(w, [4, 3], False))

(6*t^8 + t^6)*q^29 + (5*t^8 + t^6)*q^28 + (5*t^8 + t^6)*q^27 + (4*t^8 + t^6)*q^26 + (4*t^8 + t^6)*q^25 + q^20*t^6 + (3*t^8 + t^6)*q^24 + q^19*t^6 + (3*t^8 + t^6)*q^23 + q^18*t^6 + (2*t^8 + t^6)*q^22 + q^17*t^6 + (t^8 + t^6)*q^21 + q^16*t^6 + q^15*t^6 + q^14*t^6 + q^13*t^6 + q^7*t^4

In [16]:
series_t_q(list_of_list_of_partition_ends(w, [2, 1], True))

(19*t^9 + 35*t^7 + 12*t^5 + t^3)*q^29 + (13*t^9 + 32*t^7 + 11*t^5 + t^3)*q^28 + (9*t^9 + 28*t^7 + 11*t^5 + t^3)*q^27 + (5*t^9 + 25*t^7 + 10*t^5 + t^3)*q^26 + (3*t^9 + 22*t^7 + 10*t^5 + t^3)*q^25 + (t^9 + 19*t^7 + 9*t^5 + t^3)*q^24 + (16*t^7 + 9*t^5 + t^3)*q^23 + (14*t^7 + 8*t^5 + t^3)*q^22 + (11*t^7 + 8*t^5 + t^3)*q^21 + (9*t^7 + 7*t^5 + t^3)*q^20 + (7*t^7 + 7*t^5 + t^3)*q^19 + (5*t^7 + 6*t^5 + t^3)*q^18 + (3*t^7 + 6*t^5 + t^3)*q^17 + (2*t^7 + 5*t^5 + t^3)*q^16 + (5*t^5 + t^3)*q^15 + (4*t^5 + t^3)*q^14 + (4*t^5 + t^3)*q^13 + (3*t^5 + t^3)*q^12 + (3*t^5 + t^3)*q^11 + (2*t^5 + t^3)*q^10 + q^8*t^3 + (t^5 + t^3)*q^9 + q^7*t^3 + q^6*t^3 + q^5*t^3 + q^4*t^3 + q*t

In [17]:
series_t_q(list_of_list_of_partition_ends(w, [4, 1], True))

(2*t^9 + 19*t^7 + 10*t^5 + t^3)*q^29 + (17*t^7 + 9*t^5 + t^3)*q^28 + (14*t^7 + 9*t^5 + t^3)*q^27 + (12*t^7 + 8*t^5 + t^3)*q^26 + (10*t^7 + 8*t^5 + t^3)*q^25 + (8*t^7 + 7*t^5 + t^3)*q^24 + (6*t^7 + 7*t^5 + t^3)*q^23 + (5*t^7 + 6*t^5 + t^3)*q^22 + (3*t^7 + 6*t^5 + t^3)*q^21 + (2*t^7 + 5*t^5 + t^3)*q^20 + (t^7 + 5*t^5 + t^3)*q^19 + (4*t^5 + t^3)*q^18 + (4*t^5 + t^3)*q^17 + (3*t^5 + t^3)*q^16 + (3*t^5 + t^3)*q^15 + (2*t^5 + t^3)*q^14 + (2*t^5 + t^3)*q^13 + (t^5 + t^3)*q^12 + q^10*t^3 + (t^5 + t^3)*q^11 + q^9*t^3 + q^8*t^3 + q^7*t^3 + q^6*t^3 + q*t

In [18]:
series_t_q(list_of_list_of_partition_ends(w, [5, 4, 1], True))

(4*t^9 + 7*t^7 + t^5)*q^29 + (2*t^9 + 6*t^7 + t^5)*q^28 + (t^9 + 6*t^7 + t^5)*q^27 + (5*t^7 + t^5)*q^26 + (5*t^7 + t^5)*q^25 + (4*t^7 + t^5)*q^24 + (4*t^7 + t^5)*q^23 + (3*t^7 + t^5)*q^22 + (3*t^7 + t^5)*q^21 + q^17*t^5 + (2*t^7 + t^5)*q^20 + q^16*t^5 + (2*t^7 + t^5)*q^19 + q^15*t^5 + (t^7 + t^5)*q^18 + q^14*t^5 + q^13*t^5 + q^12*t^5 + q^11*t^5 + q^5*t^3

In [19]:
series_t_q(list_of_list_of_partition_ends(w, [5, 4, 1], False))

q^25*t^7 + q^24*t^7 + (3*t^9 + t^7)*q^29 + q^23*t^7 + (3*t^9 + t^7)*q^28 + q^22*t^7 + (2*t^9 + t^7)*q^27 + q^21*t^7 + (t^9 + t^7)*q^26 + q^20*t^7 + q^19*t^7 + q^18*t^7 + q^17*t^7 + q^10*t^5

In [20]:
series_t_q(list_of_list_of_partition_ends(w, [5, 3, 1], True))

(6*t^9 + 7*t^7 + t^5)*q^29 + (5*t^9 + 7*t^7 + t^5)*q^28 + (3*t^9 + 6*t^7 + t^5)*q^27 + (2*t^9 + 6*t^7 + t^5)*q^26 + (t^9 + 5*t^7 + t^5)*q^25 + (5*t^7 + t^5)*q^24 + (4*t^7 + t^5)*q^23 + (4*t^7 + t^5)*q^22 + (3*t^7 + t^5)*q^21 + (3*t^7 + t^5)*q^20 + (2*t^7 + t^5)*q^19 + q^15*t^5 + (2*t^7 + t^5)*q^18 + q^14*t^5 + (t^7 + t^5)*q^17 + q^13*t^5 + (t^7 + t^5)*q^16 + q^12*t^5 + q^11*t^5 + q^10*t^5 + q^4*t^3

In [21]:
series_t_q(list_of_list_of_partition_ends(w, [5, 3, 1], False))

(4*t^9 + t^7)*q^29 + q^23*t^7 + (3*t^9 + t^7)*q^28 + q^22*t^7 + (3*t^9 + t^7)*q^27 + q^21*t^7 + (2*t^9 + t^7)*q^26 + q^20*t^7 + (2*t^9 + t^7)*q^25 + q^19*t^7 + (t^9 + t^7)*q^24 + q^18*t^7 + q^17*t^7 + q^16*t^7 + q^9*t^5

In [22]:
series_t_q(list_of_list_of_partition_ends(w, [3, 1, 1], True))

(2*t^10 + 23*t^8 + 10*t^6 + t^4)*q^29 + (t^10 + 20*t^8 + 10*t^6 + t^4)*q^28 + (17*t^8 + 9*t^6 + t^4)*q^27 + (15*t^8 + 9*t^6 + t^4)*q^26 + (12*t^8 + 8*t^6 + t^4)*q^25 + (10*t^8 + 8*t^6 + t^4)*q^24 + (8*t^8 + 7*t^6 + t^4)*q^23 + (6*t^8 + 7*t^6 + t^4)*q^22 + (4*t^8 + 6*t^6 + t^4)*q^21 + (3*t^8 + 6*t^6 + t^4)*q^20 + (t^8 + 5*t^6 + t^4)*q^19 + (5*t^6 + t^4)*q^18 + (4*t^6 + t^4)*q^17 + (4*t^6 + t^4)*q^16 + (3*t^6 + t^4)*q^15 + (3*t^6 + t^4)*q^14 + q^11*t^4 + (2*t^6 + t^4)*q^13 + q^10*t^4 + (2*t^6 + t^4)*q^12 + q^9*t^4 + q^8*t^4 + q^7*t^4 + q^6*t^4 + q^2*t^2

In [23]:
series_t_q(list_of_list_of_partition_ends(w, [4, 1, 1], True))

(17*t^8 + 9*t^6 + t^4)*q^29 + (14*t^8 + 9*t^6 + t^4)*q^28 + (12*t^8 + 8*t^6 + t^4)*q^27 + (10*t^8 + 8*t^6 + t^4)*q^26 + (8*t^8 + 7*t^6 + t^4)*q^25 + (6*t^8 + 7*t^6 + t^4)*q^24 + (5*t^8 + 6*t^6 + t^4)*q^23 + (3*t^8 + 6*t^6 + t^4)*q^22 + (2*t^8 + 5*t^6 + t^4)*q^21 + (t^8 + 5*t^6 + t^4)*q^20 + (4*t^6 + t^4)*q^19 + (4*t^6 + t^4)*q^18 + (3*t^6 + t^4)*q^17 + (3*t^6 + t^4)*q^16 + (2*t^6 + t^4)*q^15 + (2*t^6 + t^4)*q^14 + q^11*t^4 + (t^6 + t^4)*q^13 + q^10*t^4 + (t^6 + t^4)*q^12 + q^9*t^4 + q^8*t^4 + q^7*t^4 + q^2*t^2

In [24]:
series_t_q(list_of_list_of_partition_ends(w, [4, 1, 1], False))

(2*t^10 + 6*t^8 + t^6)*q^29 + (t^10 + 6*t^8 + t^6)*q^28 + (5*t^8 + t^6)*q^27 + (5*t^8 + t^6)*q^26 + (4*t^8 + t^6)*q^25 + (4*t^8 + t^6)*q^24 + (3*t^8 + t^6)*q^23 + q^18*t^6 + (3*t^8 + t^6)*q^22 + q^17*t^6 + (2*t^8 + t^6)*q^21 + q^16*t^6 + (2*t^8 + t^6)*q^20 + q^15*t^6 + (t^8 + t^6)*q^19 + q^14*t^6 + q^13*t^6 + q^12*t^6 + q^6*t^4